In [ ]:
def rb_veldiff(lam_cen,lam_offset):    
	z=(lam_offset/lam_cen) -1.
	C = 299792.458;  #% speed of light [km/sec]
	Beta =((z+1.)**2 - 1.)/(1. + (z+1.)**2.)
	return Beta*C

def rb_doublet_abs(modelpar,wave,lam_cen,FWHM):
    vel=rb_veldiff(lam_cen[0],wave)
    #% Define first the systemic component as a double gaussian now
    A=modelpar[0]
    sig=modelpar[2]
    mu1=modelpar[1]
    mu2=rb_veldiff(lam_cen[0],lam_cen[1])
    one_minus_sym= 1.- (A*np.exp(-0.5*((vel-mu1)/sig)**2.) +A*np.exp(-0.5*((vel-mu2)/(sig))**2.))
    #% Define first the outflowing compoent
    a_out1=modelpar[3]
    DR_out=modelpar[4]
    #%width
    line_width_out1=modelpar[5]#%modelpar.outflow.width;
    line_centre_out1=mu1+modelpar[6]#;%modelpar.outflow.velcen;
    line_centre_out2=mu2+modelpar[6]#;%modelpar.outflow.velcen;
    one_minus_outflow= 1.- (DR_out*a_out1*np.exp(-0.5*((vel-line_centre_out2)/line_width_out1)**2.) +a_out1*np.exp(-0.5*((vel-line_centre_out1)/(line_width_out1))**2.));
    #% Define the inflowing compoent
    a_in=modelpar[7]
    DR_in=modelpar[8]
    #%width
    line_width_in=modelpar[9]#%modelpar.outflow.width;
    line_centre_in1=mu1+modelpar[10]#;%modelpar.outflow.velcen;
    line_centre_in2=mu2+modelpar[10]#;%modelpar.outflow.velcen;
    one_minus_inflow= 1.- (DR_in*a_in*np.exp(-0.5*((vel-line_centre_in2)/line_width_in)**2.) +a_in*np.exp(-0.5*((vel-line_centre_in1)/(line_width_in))**2.));
    #% Define  the emission component
    a_em=modelpar[11]#;%modelpar.emission.a;
    DR_em=modelpar[12]
    #%width
    line_width_em=modelpar[13]#;%modelpar.emission.width;
    line_centre_em1=mu1+modelpar[14]#;%modelpar.emission.velcen;
    line_centre_em2=mu2+modelpar[14]#;%modelpar.emission.velcen;
    fem= 1.+ (DR_em*a_em*np.exp(-0.5*((vel-line_centre_em2)/line_width_em)**2.) +a_em*np.exp(-0.5*((vel-line_centre_em1)/(line_width_em))**2.));
    final_profile=one_minus_outflow*one_minus_sym*one_minus_inflow*fem
    # Create kernel
    window_size_number_of_points=np.round(FWHM/(2.355*(vel[2]-vel[1])))
    g = Gaussian1DKernel(stddev=window_size_number_of_points)
    # Convolve data
    conv_spec =1.- convolve(1.-final_profile, g,'same')
    return conv_spec,one_minus_outflow,one_minus_sym,final_profile
  


def rb_doublet_abs_simple(modelpar,wave,lam_cen,FWHM):
    vel=rb_veldiff(lam_cen[0],wave)
    #% Define first the systemic component as a double gaussian now
    A=modelpar[0]
    sig=modelpar[2]
    mu1=modelpar[1]
    mu2=rb_veldiff(lam_cen[0],lam_cen[1])
    one_minus_sym= 1.- (A*np.exp(-0.5*((vel-mu1)/sig)**2.) +A*np.exp(-0.5*((vel-mu2)/(sig))**2.))
    #% Define first the outflowing compoent
    final_profile=one_minus_sym
    # Create kernel
    g = Gaussian1DKernel(stddev=FWHM/2.355)
    # Convolve data
    conv_spec =1.- convolve(1.-final_profile, g)
    return conv_spec,one_minus_outflow,one_minus_sym